In [1]:
#Data Loading

In [2]:
!pip install seaborn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns


In [3]:
#data CLEANING now

In [4]:
df = pd.read_csv("data/raw/Bengaluru_House_Data.csv")
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nData Types:")
print(df.dtypes)

print("\nTop 10 missing value columns:")


Shape: (13320, 9)

Columns:
['area_type', 'availability', 'location', 'size', 'society', 'total_sqft', 'bath', 'balcony', 'price']

Data Types:
area_type        object
availability     object
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

Top 10 missing value columns:


In [5]:
df

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


In [6]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [7]:
df1 = df.drop('society', axis=1)
df1.shape

(13320, 8)

In [8]:
df1['BHK'] = df1['size'].str.extract(r'(\d+)').astype(float)
df1[['size', 'BHK']].head()

,size,BHK
0,2 BHK,2.0
1,4 Bedroom,4.0
2,3 BHK,3.0
3,3 BHK,3.0
4,2 BHK,2.0


In [9]:
df1 = df1.dropna(subset=['BHK'])
df1.shape

(13304, 9)

In [10]:
df2 = df1.drop('size', axis=1)
df2.head()

,area_type,availability,location,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,19-Dec,Electronic City Phase II,1056,2.0,1.0,39.07,2.0
1,Plot Area,Ready To Move,Chikka Tirupathi,2600,5.0,3.0,120.00,4.0
2,Built-up Area,Ready To Move,Uttarahalli,1440,2.0,3.0,62.00,3.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521,3.0,1.0,95.00,3.0
4,Super built-up Area,Ready To Move,Kothanur,1200,2.0,1.0,51.00,2.0


In [11]:
df2['total_sqft'].unique()[:20]

array(['1056', '2600', '1440', '1521', '1200', '1170', '2732', '3300',
       '1310', '1020', '1800', '2785', '1000', '1100', '2250', '1175',
       '1180', '1540', '2770', '600'], dtype=object)

In [12]:
df2[df2['total_sqft'].str.contains('-', na=False)].head()

,area_type,availability,location,total_sqft,bath,balcony,price,BHK
30,Super built-up Area,19-Dec,Yelahanka,2100 - 2850,4.0,0.0,186.000,4.0
56,Built-up Area,20-Feb,Devanahalli,3010 - 3410,NaN,NaN,192.000,4.0
81,Built-up Area,18-Oct,Hennur Road,2957 - 3450,NaN,NaN,224.500,4.0
122,Super built-up Area,18-Mar,Hebbal,3067 - 8156,4.0,0.0,477.000,4.0
137,Super built-up Area,19-Mar,8th Phase JP Nagar,1042 - 1105,2.0,0.0,54.005,2.0


In [13]:
def convert_sqft(x):
    try:
        if '-' in x:
            tokens = x.split('-')
            return (float(tokens[0]) + float(tokens[1])) / 2
        return float(x)
    except:
        return None

df2['total_sqft'] = df2['total_sqft'].apply(convert_sqft)

df2['total_sqft'].dtype

dtype('float64')

In [14]:
df2['total_sqft'].isna().sum()

np.int64(46)

In [15]:
df2 = df2.dropna(subset=['total_sqft'])
df2.shape

(13258, 8)

In [16]:
df2['location'] = df2['location'].str.strip()
df2['location'].isna().sum()

np.int64(1)

In [17]:
df2 = df2.dropna(subset=['location'])
df2.shape

(13257, 8)

In [18]:
df2['location'].nunique()

1287

In [19]:
location_stats = df2['location'].value_counts()
location_stats_less_than_10 = location_stats[location_stats <= 10]

df2['location'] = df2['location'].apply(
    lambda x: 'other' if x in location_stats_less_than_10 else x
)

df2['location'].nunique()

241

In [20]:
df2['location'].value_counts().head(10)

location
other              2873
Whitefield          538
Sarjapur  Road      397
Electronic City     304
Kanakpura Road      271
Thanisandra         236
Yelahanka           212
Uttarahalli         186
Hebbal              177
Marathahalli        175
Name: count, dtype: int64

In [21]:
df2.head()

,area_type,availability,location,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Plot Area,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Built-up Area,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Super built-up Area,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [22]:
df2['price_per_sqft'] = (df2['price'] * 100000) / df2['total_sqft']
df2['price_per_sqft'].describe()

count    1.325700e+04
mean     7.912825e+03
std      1.064976e+05
min      2.678298e+02
25%      4.271186e+03
50%      5.438596e+03
75%      7.313318e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

In [23]:
df2.head()

,area_type,availability,location,total_sqft,bath,balcony,price,BHK,price_per_sqft
0,Super built-up Area,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0,3699.810606
1,Plot Area,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0,4615.384615
2,Built-up Area,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3.0,4305.555556
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0,6245.890861
4,Super built-up Area,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2.0,4250.000000


In [24]:
df2['price_per_sqft'].describe()

count    1.325700e+04
mean     7.912825e+03
std      1.064976e+05
min      2.678298e+02
25%      4.271186e+03
50%      5.438596e+03
75%      7.313318e+03
max      1.200000e+07
Name: price_per_sqft, dtype: float64

In [25]:
df2 = df2[(df2['price_per_sqft'] > 1000) & (df2['price_per_sqft'] < 50000)]
df2['price_per_sqft'].describe()

count    13232.000000
mean      6652.924808
std       4100.528024
min       1166.666667
25%       4271.722691
50%       5434.782609
75%       7305.313404
max      48484.848485
Name: price_per_sqft, dtype: float64

In [26]:
df2.shape

(13232, 9)

In [27]:
df2[df2['bath'] > df2['BHK'] + 2].shape

(14, 9)

In [28]:
df2 = df2[df2['bath'] <= df2['BHK'] + 2]
df2.shape

(13161, 9)

In [29]:
df2[df2['total_sqft'] / df2['BHK'] < 300].shape

(729, 9)

In [30]:
df2 = df2[df2['total_sqft'] / df2['BHK'] >= 300]
df2.shape

(12432, 9)

In [31]:
df2.head()

,area_type,availability,location,total_sqft,bath,balcony,price,BHK,price_per_sqft
0,Super built-up Area,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0,3699.810606
1,Plot Area,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0,4615.384615
2,Built-up Area,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3.0,4305.555556
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0,6245.890861
4,Super built-up Area,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2.0,4250.000000


In [32]:
df2 = df2.drop('price_per_sqft', axis=1)
df2.columns

Index(['area_type', 'availability', 'location', 'total_sqft', 'bath',
       'balcony', 'price', 'BHK'],
      dtype='object')

In [33]:
df2.head()

,area_type,availability,location,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Plot Area,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Built-up Area,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Super built-up Area,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [34]:
df2 = df2.drop('availability', axis=1)
df2.columns

Index(['area_type', 'location', 'total_sqft', 'bath', 'balcony', 'price',
       'BHK'],
      dtype='object')

In [35]:
df2.head()

,area_type,location,total_sqft,bath,balcony,price,BHK
0,Super built-up Area,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Plot Area,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Built-up Area,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Super built-up Area,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Super built-up Area,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [36]:
df2.isna().sum()

area_type       0
location        0
total_sqft      0
bath            0
balcony       436
price           0
BHK             0
dtype: int64

In [37]:
df2['balcony'] = df2['balcony'].fillna(0)
df2['balcony'].isna().sum()

np.int64(0)

In [38]:
#data TRAINING now

In [39]:
#First we sperate the features X and y


In [40]:
X = df2.drop(['price'],axis='columns')
y = df2['price']

In [41]:
X.head()

,area_type,location,total_sqft,bath,balcony,BHK
0,Super built-up Area,Electronic City Phase II,1056.0,2.0,1.0,2.0
1,Plot Area,Chikka Tirupathi,2600.0,5.0,3.0,4.0
2,Built-up Area,Uttarahalli,1440.0,2.0,3.0,3.0
3,Super built-up Area,Lingadheeranahalli,1521.0,3.0,1.0,3.0
4,Super built-up Area,Kothanur,1200.0,2.0,1.0,2.0


In [42]:
#As our 'X' is having Categorical Data in it and model Cannot handle text

#area_type (categorical) // Need to fix this
#location (categorical) // Need to fix this 

#total_sqft (numeric)
#bath (numeric)
#balcony (numeric)
#BHK (numeric)

# We will use One-Hot-Encoding for this

In [43]:
#One-Hot-Encoding

X_encoded = pd.get_dummies(X,columns=['area_type','location'],drop_first=True)
X_encoded = X_encoded.astype(int)
X_encoded.head()

,total_sqft,bath,balcony,BHK,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,location_1st Phase JP Nagar,location_2nd Phase Judicial Layout,location_2nd Stage Nagarbhavi,...,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_other
0,1056,2,1,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2600,5,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1440,2,3,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1521,3,1,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1200,2,1,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Now training of Model 

In [45]:
# But before that we will split our data using train_test_split

In [46]:
import sys
!{sys.executable} -m pip install -U scikit-learn

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded ,y , test_size=0.2, random_state =42 )

In [47]:
X_train.shape, X_test.shape

((9945, 247), (2487, 247))

In [48]:
# training the model (LinearRegression)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

# test model accuracy
model.score(X_test,y_test)

0.6797185192870878

In [49]:
# training the model (Ridge)

#to mitigate potential multicollinearity and overfitting

from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

ridge.score(X_test, y_test)


0.6805782067883042

In [50]:
# training the model (RandomForest)

#Handles non-linearity

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
)

rf.fit(X_train, y_train)

rf.score(X_test, y_test)

0.6487690036197301

In [51]:
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(random_state=42)
hgb.fit(X_train, y_train)

hgb.score(X_test, y_test)

0.7056559535133848

In [52]:
final_model = HistGradientBoostingRegressor(random_state=42)
final_model.fit(X_encoded, y)

,"loss loss: {'squared_error', 'absolute_error', 'gamma', 'poisson', 'quantile'}, default='squared_error'The loss function to use in the boosting process. Note that the""squared error"", ""gamma"" and ""poisson"" losses actually implement""half least squares loss"", ""half gamma deviance"" and ""half poissondeviance"" to simplify the computation of the gradient. Furthermore,""gamma"" and ""poisson"" losses internally use a log-link, ""gamma""requires ``y > 0`` and ""poisson"" requires ``y >= 0``.""quantile"" uses the pinball loss... versionchanged:: 0.23 Added option 'poisson'... versionchanged:: 1.1 Added option 'quantile'... versionchanged:: 1.3 Added option 'gamma'.",'squared_error'
,"quantile quantile: float, default=NoneIf loss is ""quantile"", this parameter specifies which quantile to be estimatedand must be between 0 and 1.",None
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.1
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees.",100
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",31
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",None
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",20
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",0.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide ` and:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_categorical.py`... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... versionchanged:: 

In [53]:
# Now We will Make export our model into File 'hgb_model.pkl'

import joblib

joblib.dump(final_model,'hgb_model.pkl')
joblib.dump(list(X_encoded.columns), "model_columns.pkl")

['model_columns.pkl']

In [54]:
import os
os.path.getsize("hgb_model.pkl"), os.path.getsize("model_columns.pkl")

(126470, 5986)

In [55]:
####################################################################################################

In [56]:
# Predict Function
import pandas as pd
import joblib

# Load artifacts
model = joblib.load("hgb_model.pkl")
columns = joblib.load("model_columns.pkl")

def predict_price(area_type, location, sqft, bath, balcony, bhk):
    # Create input dataframe
    input_dict = {
        "area_type": area_type,
        "location": location,
        "total_sqft": sqft,
        "bath": bath,
        "balcony": balcony,
        "BHK": bhk
    }
    
    input_df = pd.DataFrame([input_dict])
    
    # One-hot encode
    input_encoded = pd.get_dummies(input_df)
    
    # Reindex to match training columns
    input_encoded = input_encoded.reindex(columns=columns, fill_value=0)
    
    return model.predict(input_encoded)[0]

In [57]:
pred = predict_price(
    area_type="Super built-up Area",
    location="Whitefield",
    sqft=1200,
    bath=2,
    balcony=1,
    bhk=2
)

pred

np.float64(65.32519280004834)

In [58]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

def predict_from_payload(payload: dict) -> dict:
    """
    Expected payload keys:
      - area_type (str)
      - location (str)
      - total_sqft (float or int)
      - bath (float or int)
      - balcony (float or int)
      - BHK (float or int)
    Returns:
      - predicted_price_lakhs (float)
    """
    pred = predict_price(
        area_type=payload["area_type"],
        location=payload["location"],
        sqft=float(payload["total_sqft"]),
        bath=float(payload["bath"]),
        balcony=float(payload["balcony"]),
        bhk=float(payload["BHK"]),
    )
    return {"predicted_price_lakhs": float(pred)}


In [59]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

test_payload = {
    "area_type": "Super built-up Area",
    "location": "Whitefield",
    "total_sqft": 1200,
    "bath": 2,
    "balcony": 1,
    "BHK": 2
}

predict_from_payload(test_payload)

{'predicted_price_lakhs': 65.32519280004834}

In [60]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

def validate_payload(payload: dict) -> None:
    required = ["area_type", "location", "total_sqft", "bath", "balcony", "BHK"]
    missing = [k for k in required if k not in payload]
    if missing:
        raise ValueError(f"Missing required fields: {missing}")

    # Basic type + range checks
    sqft = float(payload["total_sqft"])
    bath = float(payload["bath"])
    balcony = float(payload["balcony"])
    bhk = float(payload["BHK"])

    if sqft <= 0:
        raise ValueError("total_sqft must be > 0")
    if bhk <= 0:
        raise ValueError("BHK must be > 0")
    if bath <= 0:
        raise ValueError("bath must be > 0")
    if balcony < 0:
        raise ValueError("balcony must be >= 0")

    if not isinstance(payload["area_type"], str) or not payload["area_type"].strip():
        raise ValueError("area_type must be a non-empty string")
    if not isinstance(payload["location"], str) or not payload["location"].strip():
        raise ValueError("location must be a non-empty string")


def predict_from_payload_safe(payload: dict) -> dict:
    try:
        validate_payload(payload)
        result = predict_from_payload(payload)
        return {"ok": True, **result}
    except Exception as e:
        return {"ok": False, "error": str(e)}

In [61]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

# good
predict_from_payload_safe({
    "area_type": "Super built-up Area",
    "location": "Whitefield",
    "total_sqft": 1200,
    "bath": 2,
    "balcony": 1,
    "BHK": 2
})

{'ok': True, 'predicted_price_lakhs': 65.32519280004834}

In [62]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

# bad (missing sqft)
predict_from_payload_safe({
    "area_type": "Super built-up Area",
    "location": "Whitefield",
    "bath": 2,
    "balcony": 1,
    "BHK": 2
}

SyntaxError: incomplete input (1054177731.py, line 13)

In [ ]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################


import json

def lambda_handler(event, context=None):
    """
    Expects API Gateway proxy event:
      event["body"] is a JSON string with the payload.
    Returns:
      { "statusCode": int, "headers": {...}, "body": json-string }
    """
    try:
        body = event.get("body", "{}")
        if isinstance(body, str):
            payload = json.loads(body)
        else:
            payload = body  # in case body already parsed

        result = predict_from_payload_safe(payload)

        status_code = 200 if result.get("ok") else 400
        return {
            "statusCode": status_code,
            "headers": {"Content-Type": "application/json"},
            "body": json.dumps(result)
        }
    except Exception as e:
        return {
            "statusCode": 500,
            "headers": {"Content-Type": "application/json"},
            "body": json.dumps({"ok": False, "error": f"Internal error: {str(e)}"})
        }

In [ ]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################


test_event = {
    "body": json.dumps({
        "area_type": "Super built-up Area",
        "location": "Whitefield",
        "total_sqft": 1200,
        "bath": 2,
        "balcony": 1,
        "BHK": 2
    })
}

lambda_handler(test_event)

In [ ]:
import os
os.path.exists("requirements.txt")

In [ ]:
import os
os.getcwd()